In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import time
import pandas as pd
import streamlit as st
import altair as alt

from bizlogic_corr_risknorm_vix import (
    load_df_riskvix,
    load_basel_risknorms,
    calculate_risk_score,
    generate_ui_df,
    calculate_corr,
    convert_diplay_df,
)

from streamlit_jupyter import StreamlitPatcher, tqdm

In [3]:
df_riskvix = load_df_riskvix("df_all.parquet")
df_baselrisknorms = load_basel_risknorms(csvfile="risk_mean_basel_risk_norms.csv")

df_ui = None
prv_riskword_score_threshold = None

In [16]:
riskword_score_threshold = '40'

lst_freq = ["D (daily)", "W (Weekly)", "M (Monthly)"]
rollup_freq = lst_freq[2]

riskword_score_threshold = int(riskword_score_threshold)

In [17]:
if df_ui is None or prv_riskword_score_threshold != riskword_score_threshold:
    print(f'calculate_risk_score: riskword_score_threshold={riskword_score_threshold}')
    df_riskvix = calculate_risk_score(
        df_riskvix, df_baselrisknorms, riskword_score_threshold
    )

In [18]:
target_freq = rollup_freq.split(" ")[0]

In [62]:
df_ui = generate_ui_df(df_riskvix, target_freq)

In [63]:
prv_riskword_score_threshold = riskword_score_threshold

corr_val = calculate_corr(df_ui)
print("Correlation: " + str(corr_val))

Correlation: 0.918


In [68]:
cht_base = alt.Chart(df_ui).encode(alt.X("pubdate:T", axis=alt.Axis(title="Date")))

line_vix = cht_base.mark_line(stroke="#57A44C", point=alt.OverlayMarkDef(color="#57A44C")).encode(
    alt.Y("VIXCLS").title("VIX index")
)
line_riskscore = cht_base.mark_line(
    stroke="#5276A7", point=alt.OverlayMarkDef(color="#5276A7")
).encode(alt.Y("risk_score_scaled").title("Risk score"))

In [69]:
alt.layer(line_vix, line_riskscore).resolve_scale(y="independent").configure_axisLeft(titleColor='#57A44C').configure_axisRight(titleColor="#5276A7")

alt.LayerChart(...)

In [67]:
StreamlitPatcher().jupyter() 

st.dataframe(convert_diplay_df(df_ui))

,pubdate,mean_risk_score,news_number,VIXCLS
0,2018-01-01,5.392757e+06,23564,345.84
1,2018-02-01,6.504788e+06,25671,646.09
2,2018-03-01,6.674607e+06,24678,595.86
3,2018-04-01,6.362494e+06,25405,548.44
4,2018-05-01,7.006898e+06,26314,443.32
5,2018-06-01,3.850057e+03,16,15.43
